# The BB84 Protocol

In [208]:
! pip install qiskit
! pip install --upgrade qiskit

! pip install qiskit_aer
! pip install --upgrade qiskit_aer

In [231]:
import qiskit
import qiskit_aer
import qiskit_ibm_runtime
import random

print(f'Qiskit version: {qiskit.__version__}')
print(f'Qiskit Aer version: {qiskit_aer.__version__}')
print(f'Qiskit IBM Runtime version: {qiskit_ibm_runtime.__version__}')

# Global constants
MY_IBM_TOKEN = '3eb36505b6ec871fd498923a1485b7f25ac180724da3824de0da6af96ca09f9810b958b868839edffb0d1df1c068965b49a0b8a576c473afbf0a0c3d00596371'

Qiskit version: 1.2.1
Qiskit Aer version: 0.15.1
Qiskit IBM Runtime version: 0.29.0


In [238]:
def is_bitstring(s: str) -> bool:
    return all([c == '0' or c == '1' for c in s])

def generate_random_bitstring(n: int) -> str:
    r'''
    Generates a random bitstring of length n.
    '''
    return format(random.randint(0, 2 ** n - 1), f'0{n}b')

def encode(n: int, a: str, b: str) -> qiskit.QuantumCircuit:
    r'''
    Generate quantum state `\ket{\psi}` which encodes bitstrings `a` and `b`.
    This is represented by the tensor product of the bits `a_i` (one-hot) encoded in the basis specified by `b_i`.
    Bit `a_i` is encoded using the computational basis if `b_i = 0`.
    Otherwise, if `b_i = 1`, `a_i` is encoded in the Hadamard basis.

    Parameters
    ----------
    n (int): length of bitstrings `a` and `b`
    a (str): bitstring representing the message to encode
    b (str): bitstring representing the sequence of bases in which to encode `a`

    Returns
    -------
    (QuantumCircuit): the quantum circuit containing the quantum state encoding `a` using the bases specified by `b`
    '''
    assert n == len(a) == len(b)
    assert is_bitstring(a) and is_bitstring(b)

    qc = qiskit.QuantumCircuit(n, n, name='BB84 encoding qstate')

    qc.barrier(label='Encoding >')  # barriers highlight the separate stages of the circuit
    for i in range(n):
        # Encode a_i in qubit i
        if a[i] == '1':
            qc.x(i)
        # Basis to encode a_i: computational (b_i = 0) or Hadamard (b_i = 1)
        if b[i] == '1':
            qc.h(i)
    qc.barrier(label='< Encoding')

    return qc

def decode(n: int, qc: qiskit.QuantumCircuit, b: str) -> str:
    assert n == len(b)
    assert is_bitstring(b)

    # If b_i = 0, measure in the computational basis.
    # Else, if b_i = 1, measure in the Hadamard basis.
    qc.barrier(label='Decoding >')
    for i in range(n):
        if b[i] == '1':
            qc.h(i)
    qc.measure_all(add_bits=False)
    qc.barrier(label='< Decoding')

    counts = simulate_circuit(qc, nshots=1)
    # Reverse the order of bitstrings in `counts`.
    counts = list((k[::-1], v) for k, v in counts.items())

    # It is expected that exactly one bitstring is received as output in `counts`.
    # Therefore, only the key of the first map entry is considered.
    a, _ = counts[0]

    assert n == len(a)
    assert is_bitstring(a)

    return a

def simulate_circuit(circuit, nshots=1, with_noise=False):

    qiskit_ibm_runtime.QiskitRuntimeService.save_account(
        token=MY_IBM_TOKEN,
        channel='ibm_cloud',
    )
    service = qiskit_ibm_runtime.QiskitRuntimeService(
        token=MY_IBM_TOKEN,
        channel='ibm_cloud',
    )
    real_backend = service.backend("ibm_brisbane")

    if with_noise == True:
        simulator = qiskit_aer.AerSimulator.from_backend(real_backend)
    else:
        simulator = qiskit_aer.AerSimulator()

    compiled_circuit = qiskit.transpile(circuit, simulator)
    sim_result       = simulator.run(compiled_circuit, shots=nshots).result()
    counts           = sim_result.get_counts()

    return counts

# def execute_circuit(circuit):
#     service = qiskit_ibm_runtime.QiskitRuntimeService()
#     backend = service.backend('ibm_brisbane')
#     pass_manager = qiskit.transpiler.generate_preset_pass_manager(optimization_level=3, backend=backend)
#     isa_circuit = pass_manager.run(circuit)

#     isa_circuit.draw('mpl')
#     ...


In [239]:
# Create a data class for the parties involved in the BB84 protocol
class Entity:
    name: str
    n: int
    a: str
    b: str
    qc: qiskit.QuantumCircuit

    def __init__(self, name=None, n=None, a=None, b=None) -> None:
        self.name = name
        self.n, self.a, self.b = n, a, b

    def __str__(self) -> str:
        return f'{self.name}: n = {self.n}, a = {self.a}, b = {self.b}'


# 1. Initial step
# Establish parties involved in the BB84 protocol:
#   * Alice: the sender
#   * Bob: the receiver
#   * Eve: the malicious third-party tapped into Alice's and Bob's communication channel
alice, bob, eve = Entity('Alice'), Entity('Bob'), Entity('Eve')
# Set length of bitstrings
alice.n = bob.n = eve.n = 8
# Set bitstrings
alice.a = generate_random_bitstring(alice.n)
alice.b = generate_random_bitstring(alice.n)
bob.b   = generate_random_bitstring(bob.n)
eve.b   = generate_random_bitstring(eve.n)
# bob.b   = alice.b
# eve.b   = alice.b
print(alice)


# 2. Sender encoding step
alice.qc = encode(alice.n, alice.a, alice.b)
# print(alice.qc)


# 3. Distribution step
# Send Alice's quantum state (contained in a quantum circuit) to Bob ...
bob.qc   = alice.qc
# ... but Eve intercepts the quantum state that Alice messaged to Bob
eve.qc   = alice.qc


# 4. Receiver decoding step
bob.a = decode(bob.n, bob.qc, bob.b)
print(bob)


# 5. Eavesdropper decoding step
eve.a = decode(eve.n, eve.qc, eve.b)
print(eve)


# 6. Sifting step
# Incorrectly decoded bits of `a` are marked with an `'x'`.
alice.a, bob.a, eve.a = list(alice.a), list(bob.a), list(eve.a)
for i in range(alice.n):
    bob.a[i] = bob.a[i] if alice.b[i] == bob.b[i] else 'x'
    eve.a[i] = eve.a[i] if alice.b[i] == eve.b[i] else 'x'
alice.a, bob.a, eve.a = ''.join(alice.a), ''.join(bob.a), ''.join(eve.a)
print()
print(alice)
print(bob)
print(eve)


# 7. Alice picks `k/2` random bits of `a` from the total of `k` bits of `a` correctly decoded
picks = list(enumerate(filter(lambda c: c == '0' or c == '1', list(bob.a))))
picks = random.sample(picks, k = len(picks) // 2)
picks = sorted(picks, key = lambda x: x[0])
picks = list(map(lambda x: x[1], picks))
picks


# 8. Pick keys
alice.key = ...
bob.key   = ...



Alice: n = 8, a = 00100000, b = 00111111


InvalidAccountError: "Invalid `instance` value. Expected a non-empty string, got 'None'. If using the ibm_quantum channel, please specify the channel when saving your account with `channel = 'ibm_quantum'`."